<center><h1>Least-Squares Support Vector machine</h1></center>

In [1]:
# loading dataset
%run -i 'load_dataset.py'

In [55]:
import numpy as np

# printing datasets info
print("{:10}{:18}{:}".format(
        'Dataset:',
        'Features.shape:',
        '# of classes:',
        ))
for dataset_name, data in datasets.items():
    print("{:9} {:17} {:}".format(
        dataset_name, 
        str(data['features'].shape),
        len(np.unique(data['labels'].values, axis=0))
        ))

Dataset:  Features.shape:   # of classes:
vc2c      (310, 6)          2
vc3c      (310, 6)          3
wf24f     (5456, 24)        4
wf4f      (5456, 4)         4
wf2f      (5456, 2)         4
pk        (195, 22)         2


In [5]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
# Função para mudar a escala dos dados
def scale_feat(X_train, X_test, scaleType='min-max'):
    if scaleType=='min-max' or scaleType=='std':
        X_tr_norm = np.copy(X_train) # fazendo cópia para deixar original disponível
        X_ts_norm = np.copy(X_test)
        scaler = MinMaxScaler() if scaleType=='min-max' else StandardScaler()
        scaler.fit(X_tr_norm)
        X_tr_norm = scaler.transform(X_tr_norm)
        X_ts_norm = scaler.transform(X_ts_norm)
        return (X_tr_norm, X_ts_norm)
    else:
        raise ValueError("Tipo de escala não definida. Use 'min-max' ou 'std'.")

In [594]:
# convert dummies to multilabel
def dummie_to_multilabel(X):
    N = len(X)
    X_multi = np.zeros((N,1),dtype='int')
    for i in range(N):
        temp = np.where(X[i]==1)[0] # find where 1 is found in the array
        if temp.size == 0: # is a empty array, there is no '1' in the X[i] array
            X_multi[i] = 0 # so we denote this class '0'
        else:
            X_multi[i] = temp[0] + 1 # we have +1 because 
    return X_multi.T[0]

# Class implementation of LS-SVM

In [603]:
#import plotly.offline as plt
#import plotly.graph_objs as go
#plt.init_notebook_mode(connected=True) # enabling plotly inside jupyter notebook

import numpy as np
from numpy import dot, exp
from functools import partial 
from scipy.spatial.distance import cdist

class LSSVM:
    'Class the implements the Least-Squares Support Vector machine.'
    
    def __init__(self, kernel='rbf', **kernel_params): 
        self.alpha = None
        
        self.y        = None
        self.y_labels = None
        
        self.x = None
        self.b = None
        self.kernel = LSSVM.get_kernel(kernel, **kernel_params) # saving kernel function
        
           
    @staticmethod
    def get_kernel(name, **params):
        
        def linear(x_i, x_j):                           
            return dot(x_i, x_j.T)
        
        def poly(x_i, x_j, d=params.get('d',3)):        
            return ( dot(x_i, x_j.T) + 1 )**d
        
        def rbf(x_i, x_j, sigma=params.get('sigma',1)):
            if x_i.ndim==x_i.ndim and x_i.ndim==2: # both matrices
                return exp( -cdist(x_i,x_j)**2 / sigma**2 )
#             return exp( -( dot(x_i,x_i.T) + dot(x_j,x_j.T)- 2*dot(x_i,x_j) ) / sigma**2 )
#             temp = x_i.T - X
#             return exp( -dot(temp.temp) / sigma**2 )
                
        kernels = {'linear': linear, 'poly': poly, 'rbf': rbf}
                
        if kernels.get(name) is None: 
            raise KeyError("Kernel '{}' is not defined, try one of the list: {}.".format(
                name, list(kernels.keys())))
        else: return kernels[name]
        
    
    def opt_params(self, X, y_values, gamma):
        sigma = np.multiply( y_values*y_values.T, self.kernel(X,X) )

        A_cross = np.linalg.pinv(np.block([
            [0,                      y_values.T                   ],
            [y_values,   sigma + gamma**-1 * np.eye(len(y_values))]
        ]))

        B = np.array([0]+[1]*len(y_values))

        solution = dot(A_cross, B)
        b     = solution[0]
        alpha = solution[1:]
        
        return (b, alpha)
            
    
    def fit(self, X, Y, gamma, verboses=0):
        self.x = X
        self.y = Y
        self.y_labels = np.unique(Y, axis=0)
        
        if len(self.y_labels)==2: # binary classification
            # converting to -1/+1
            y_values = np.where(Y==self.y_labels[0],-1,1)
            self.b, self.alpha = self.opt_params(X, y_values, gamma)
        
        else: # multiclass classification
              # ONE-VS-ALL APPROACH
            n_classes = len(self.y_labels)
            self.b      = np.zeros(n_classes)
            self.alpha = np.zeros((n_classes, len(Y)))
            for i in range(len(self.y_labels)):
                # converting to -1/+1
                y_values = np.where((Y == self.y_labels[i]).all(axis=1),
                                    +1, -1)[:,np.newaxis] # making it a column vector
  
                self.b[i], self.alpha[i] = self.opt_params(X, y_values, gamma)

        
    def predict(self,X):
        k = self.kernel(self.x,X)
        
        if len(self.y_labels)==2: # binary classification
            y_values = np.where(self.y==self.y_labels[0],-1,1)
            Y = np.sign( dot( np.multiply(self.alpha, y_values.flatten()), k ) + self.b)
            
            y_pred_labels = np.where(Y==-1,         y_labels[0],
                                     np.where(Y==1,y_labels[1],Y))
        
        else: # multiclass classification, ONE-VS-ALL APPROACH
            Y = np.zeros((len(self.y_labels), len(X)))
            for i in range(len(self.y_labels)):
                y_values = np.where((self.y == self.y_labels[i]).all(axis=1),
                                         +1, -1)[:,np.newaxis] # making it a column vector
                Y[i] = np.sign( dot( np.multiply(self.alpha[i], y_values.flatten()), k ) + self.b[i])
            
            predictions = np.argmax(Y, axis=0)
            y_pred_labels = np.array([self.y_labels[i] for i in predictions])
            
        return y_pred_labels
        

#########################################################################################################

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


for dataset_name in datasets:
    print(dataset_name)

    X = datasets[dataset_name]['features'].values
    Y = datasets[dataset_name]['labels'].values

    X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.2)  # Train/Test split = 80%/20%
    X_tr_norm, X_ts_norm = scale_feat(X_train, X_test, scaleType='min-max') # scaling features
    
    print('linear')
    lssvm = LSSVM(kernel='linear')
    lssvm.fit(X_tr_norm, y_train, gamma=1)
    print(accuracy_score(dummie_to_multilabel(y_test), 
                         dummie_to_multilabel(lssvm.predict(X_ts_norm))))
    
    print('poly')
    lssvm = LSSVM(kernel='poly', d=3)
    lssvm.fit(X_tr_norm, y_train, gamma=1)
    print(accuracy_score(dummie_to_multilabel(y_test), 
                         dummie_to_multilabel(lssvm.predict(X_ts_norm))))
    
    print('rbf')
    lssvm = LSSVM(kernel='rbf', sigma=1)
    lssvm.fit(X_tr_norm, y_train, gamma=1)
    print(accuracy_score(dummie_to_multilabel(y_test), 
                         dummie_to_multilabel(lssvm.predict(X_ts_norm))))
    
    print('\n','#'*100,'\n')

vc2c
linear
0.7258064516129032
poly
0.8548387096774194
rbf
0.7903225806451613

 #################################################################################################### 

vc3c
linear
0.8548387096774194
poly
0.8387096774193549
rbf
0.8387096774193549

 #################################################################################################### 

wf24f
linear
0.6263736263736264
poly
0.9001831501831502
rbf
0.9056776556776557

 #################################################################################################### 

wf4f
linear
0.7417582417582418
poly
0.771978021978022
rbf
0.7701465201465202

 #################################################################################################### 

wf2f
linear
0.7243589743589743
poly
0.7335164835164835
rbf
0.7371794871794872

 #################################################################################################### 

pk
linear
0.8205128205128205
poly
0.8205128205128205
rbf
0.8461538461538461

 #######